In [1]:
import pandas as pd
import numpy as np
import math
import sys, os
import sklearn.metrics


In [8]:
res_dir='run75'
all_results= {'full_methods': [res_dir+'/full',2],
    'svote': [res_dir+'/svote',3],
    't2_rf16': [res_dir+'/t2/16xcept',2],
    't2_svm16': [res_dir+'/t2/16xcept',4],
    't2_wdnn16': [res_dir+'/t2/16xcept',6],
#     't2_rf21': [res_dir+'/t2/21xcept',2],
#     't2_svm21':[res_dir+'/t2/21xcept',4],
#     't2_wdnn21':[res_dir+'/t2/21xcept',6],
    'softmax16':[res_dir+'/extras/closedsoftmax_breakdown',2],
    'openset21':[res_dir+'/extras/openset_breakdown',2]}
FOLDS=[1,2,3,4,5]
SEEDS=[0,1,2,3,4]
comparison_dict={}


In [11]:
for root,dirs,files in os.walk(all_results['full_methods'][0],all_results['full_methods'][1]):
    for f in files:
        if f.split('.')[-1]=='csv':
            seed = f.split('seed')[-1][0]
            fold = f.split('fold')[-1][0]
            name='f{}s{}'.format(fold,seed)
            comparison_dict[name] = pd.read_csv('{}/{}'.format(root,f),usecols = ['Id','y','preds'])
            comparison_dict[name].rename(columns={'y': 'labels', 'preds': 'Full_Methods'}, inplace=True)
for method in all_results:
    if method == 'full_methods':
        continue
    for root,dirs,files in os.walk(all_results[method][0]):
        for f in files:
            if f.split('.')[-1]=='csv':
                seed = f.split('seed')[-1][0]
                fold = f.split('fold')[-1][0]
                name='f{}s{}'.format(fold,seed)
                tmp_df = pd.read_csv('{}/{}'.format(root,f))
                comparison_dict[name][method] = tmp_df.iloc[:,all_results[method][1]] 
    print(res_dir,method)



comparison_dict['f1s0']

run75 svote
run75 t2_rf16
run75 t2_svm16
run75 t2_wdnn16
run75 softmax16
run75 openset21


,Id,labels,Full_Methods,svote,t2_rf16,t2_svm16,t2_wdnn16,softmax16,openset21
0,/opt/ImageBase/AWS_sync/cropped/aedes/aegypti/...,0,0,0,0,0,0,0,0
1,/opt/ImageBase/AWS_sync/cropped/aedes/aegypti/...,0,0,0,0,0,0,0,0
2,/opt/ImageBase/AWS_sync/cropped/aedes/aegypti/...,0,0,0,0,0,0,0,0
3,/opt/ImageBase/AWS_sync/cropped/aedes/aegypti/...,0,0,0,0,0,0,0,0
4,/opt/ImageBase/AWS_sync/cropped/aedes/aegypti/...,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1721,/opt/ImageBase/AWS_sync/cropped/uranotaenia/sa...,1,1,1,1,1,1,1,0
1722,/opt/ImageBase/AWS_sync/cropped/uranotaenia/sa...,1,1,1,1,1,1,1,0
1723,/opt/ImageBase/AWS_sync/cropped/uranotaenia/sa...,1,1,1,1,1,1,1,0
1724,/opt/ImageBase/AWS_sync/cropped/uranotaenia/sa...,1,1,1,1,1,1,1,0


In [16]:
with pd.ExcelWriter('McNemars_Analysis.xlsx') as writer:
    for name in comparison_dict.keys():
        comparison_dict[name].to_excel(writer, sheet_name=name)

In [18]:
# basic stats
stats={}
folds=comparison_dict.keys()
num_folds=len(folds)
for fold in comparison_dict.keys():
    for method in comparison_dict[fold].columns:
        if method in ['Id','labels']:
            continue
        if method not in stats:
            stats[method]={
                           'avg':{'precision':0,'recall':0,'specificity':0,'f1_score':0,'accuracy':0},
                           'stddev':{'precision':0,'recall':0,'specificity':0,'f1_score':0,'accuracy':0}}
        tn, fp, fn, tp = sklearn.metrics.confusion_matrix(comparison_dict[fold].labels.values, comparison_dict[fold][method]).ravel()
        stats[method][fold]={}
        stats[method][fold]['precision'] = tp/(tp+fp)
        stats[method][fold]['recall'] = tp/(tp+fn)
        stats[method][fold]['specificity'] = tn/(tn+fp)
        stats[method][fold]['f1_score'] = 2*stats[method][fold]['precision']*stats[method][fold]['recall']/(stats[method][fold]['precision']+stats[method][fold]['recall'])
        stats[method][fold]['accuracy'] = (tp+tn)/(tp+tn+fp+fn)
        
        stats[method]['avg']['precision']+= tp/(tp+fp)
        stats[method]['avg']['recall'] += tp/(tp+fn)
        stats[method]['avg']['specificity'] += tn/(tn+fp)
        stats[method]['avg']['f1_score'] += 2*stats[method][fold]['precision']*stats[method][fold]['recall']/(stats[method][fold]['precision']+stats[method][fold]['recall'])
        stats[method]['avg']['accuracy'] += (tp+tn)/(tp+tn+fp+fn)

for method in stats:
    stats[method]['avg']['precision']/=num_folds
    stats[method]['avg']['recall'] /=num_folds
    stats[method]['avg']['specificity'] /=num_folds
    stats[method]['avg']['f1_score'] /=num_folds
    stats[method]['avg']['accuracy'] /=num_folds
    
for method in stats:
    for fold in folds:
#         print(fold,method)
#         print(stats[method]['stddev']['precision'])
#         print(stats[method]['avg']['precision'])
#         print(stats[method][fold]['precision'])
        
        stats[method]['stddev']['precision'] += (stats[method][fold]['precision'] - stats[method]['avg']['precision'])**2  
        stats[method]['stddev']['recall'] += (stats[method][fold]['recall'] - stats[method]['avg']['recall'])**2 
        stats[method]['stddev']['specificity'] += (stats[method][fold]['specificity'] - stats[method]['avg']['specificity'])**2 
        stats[method]['stddev']['f1_score'] +=(stats[method][fold]['f1_score'] - stats[method]['avg']['f1_score'])**2 
        stats[method]['stddev']['accuracy'] +=(stats[method][fold]['accuracy'] - stats[method]['avg']['accuracy'])**2 

    
    stats[method]['stddev']['precision']=math.sqrt(stats[method]['stddev']['precision']/(num_folds-1))
    stats[method]['stddev']['recall'] =math.sqrt(stats[method]['stddev']['recall']/(num_folds-1))
    stats[method]['stddev']['specificity'] =math.sqrt(stats[method]['stddev']['specificity']/(num_folds-1))
    stats[method]['stddev']['f1_score'] =math.sqrt(stats[method]['stddev']['f1_score'] /(num_folds-1))
    stats[method]['stddev']['accuracy'] =math.sqrt(stats[method]['stddev']['accuracy'] /(num_folds-1))
    
# print(stats)

In [22]:
for method in stats:
    print(method)
    print('avg',stats[method]['avg'])
    print('std dev',stats[method]['stddev'])
    print('\n')

Full_Methods
avg {'precision': 0.9420692795409755, 'recall': 0.9218223449819445, 'specificity': 0.8079012316923178, 'f1_score': 0.93042108432081, 'accuracy': 0.8950396844859975}
std dev {'precision': 0.037555966614743926, 'recall': 0.06343045152060785, 'specificity': 0.0731839142504961, 'f1_score': 0.04007099906416844, 'accuracy': 0.056300657675393856}


svote
avg {'precision': 0.9663465665925404, 'recall': 0.8762095607801296, 'specificity': 0.898093869921305, 'f1_score': 0.9175483710686716, 'accuracy': 0.8800872879263292}
std dev {'precision': 0.022458480839474392, 'recall': 0.0719251495939653, 'specificity': 0.03923713144518885, 'f1_score': 0.04436737590549413, 'accuracy': 0.05886717163529973}


t2_rf16
avg {'precision': 0.9589065359971464, 'recall': 0.8607937095950524, 'specificity': 0.8692473716917911, 'f1_score': 0.9056918999060456, 'accuracy': 0.8618906204511602}
std dev {'precision': 0.023579773400107886, 'recall': 0.06840295553891881, 'specificity': 0.05407798889933923, 'f1_sco